## Simple Language Model

This Notebook focuses on creating a language model based on Shakespearean text. It involves several key steps, including data loading, tokenization, model creation, loss estimation, and training.

## Imports

In [20]:
import pandas as pd
import tqdm as notebook_tqdm
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

## Data

We will use the [tiny Shakespeare dataset](https://github.com/karpathy/ng-video-lecture/blob/master/input.txt). 

Read the contents of the file into a string

In [21]:
with open("tiny_shakespeare_dataset.txt", "r") as file:
    text = file.read()


Print the first 200 characters of the string for verification

In [22]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


Here are all the unique characters that occur in this text

In [23]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Tokenization: simple mapping from characters to integers. Google uses [SentencePiece](https://github.com/google/sentencepiece) and OpenAI uses [tiktoken](https://github.com/openai/tiktoken).

In [24]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of ints
decode = lambda l: "".join(itos[i] for i in l) # decode: take a list of ints, output a string

print(encode("Hello World!")) # list of integers representing that string
print(decode(encode("Hello World!"))) # decoding list above back to characters

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 2]
Hello World!


Tokenize the entire trainingset

In [25]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:200]) # print the 200 characters again but the tokenized version 

torch.Size([1115394]) <built-in method type of Tensor object at 0x13f207060>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


Split dataset into training and validation sets

In [26]:
n = int(0.9 * len(data))
train_data = data[: n]
val_data = data[n:]

print(len(train_data), len(val_data))

1003854 111540


We train the transformer on blocks of data and we also train the transformer on batches to utilize parallelization.

In [27]:
block_size = 8
batch_size = 4
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [28]:
def get_batch(split):
    # generate a small batch of data of input x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # because last will start from -8 and go until the end of text
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


## Model

Define the Bigram Language Model using PyTorch

In [29]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [30]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(5.0364, grad_fn=<NllLossBackward0>)


Define a function to estimate the loss

In [31]:
eval_iters = 200

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

## Training

We see that the model now generates random characters, so we need to train it

In [32]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist())) # generate works with batches, we take out zero-th row



lfJeukRuaRJKXAYtXzfJ:HEPiu--sDioi;ILCo3pHNTmDwJsfheKRxZCFs
lZJ XQc?:s:HEzEnXalEPklcPU cL'DpdLCafBheH


Define batch size and optimizer

In [33]:
batch_size = 32
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)


Train the model using training data and evaluate the loss

In [34]:
for iter in range(20000):

    # every once in a while evaluate the loss on train and val sets
    if iter % 2000 == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 4.6433, val loss 4.6455
step 2000: train loss 3.1202, val loss 3.1356
step 4000: train loss 2.6328, val loss 2.6573
step 6000: train loss 2.5305, val loss 2.5378
step 8000: train loss 2.4785, val loss 2.5123
step 10000: train loss 2.4654, val loss 2.4898
step 12000: train loss 2.4569, val loss 2.4805
step 14000: train loss 2.4634, val loss 2.4856
step 16000: train loss 2.4535, val loss 2.4880
step 18000: train loss 2.4556, val loss 2.4855


Generate from the model

In [35]:
context = torch.zeros((1, 1), dtype=torch.long)#, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


BENINENendede Gletthess ilike hase.

wase nt t wam o y me, lat, chund a ad,
HUMy t:
Naronofee thold is
Withat dis seshe wanoalowe:
Fan t,
Shat,
Whine thyo head
Asurg.
Pant they HAndou ammbl KINourouse y If blcortun, risear sapl wereso y y.
I'thowat worditcounte sowe toucesthesh mel he theaim thee w! tirk ar d wannchievery, wajoul:
GAUCKENomm,
BHe fou ff mendisid! sonofe Iff o weathatherd anoowowhe IXEERD ingethethof hr by urf a se llovat ninved ifut fasort keinth ayely myo MOuns batr h matovend 
